# Чтение и грубая очистка данных

In [128]:
import pandas as pd
import numpy as np
import string, re
from collections import Counter

from src.data.read_raw_data import read_from_gsheet, drop_unwanted_data, split_types

In [133]:
# Читаем данные из таблицы, удаляем неинтересующие столбцы, фильтруем по типу задания
raw_data = read_from_gsheet()
_, email_data = split_types(raw_data)

C:\Users\max_a\AppData\Local\Temp\ipykernel_2956\1533711228.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  raw_data = read_from_gsheet()


In [134]:
 # Далее будем работать с данными по письмам
email_data.head()

,Type,Question id,Question,Text,Solving a communicative task,Text structure,Use of English (for emails),Comments,Overall_score,origin,task_image,score_image
2,Email,,…I am so happy that summer has come and we are...,"Dear Ronny, I was glad to hear from you again....",1,2,2,К1 – 1 (отсутствует третий аспект). Лексико-гр...,5,ФИПИ,,
3,Email,,…I am learning to cook from my mother now. But...,"Dear Mary, Thanks for your message. It was gre...",1,2,2,По критерию «Решение коммуникативной задачи»: ...,5,ФИПИ,,
4,Email,,…I am so happy that summer has come and we are...,Moscow\n15 october\nHi!\nThanks you for you re...,1,0,0,По ОТ допущены следующие нарушения.\r\n1. Указ...,1,ФИПИ,,
5,Email,,…I’ve recently been involved in a school surve...,"Hey, Mike.\r\nHow's it going? As for me, i'm p...",0,0,0,Подведём итог по критерию «Решение коммуникати...,0,ФИПИ,,
6,Email,,…All of my friends think camping is a perfect ...,"Hi Emily,\r\nThank you for the e-mail. I'm so ...",2,1,0,Подведём итог по критерию «Решение коммуникати...,3,ФИПИ,,


In [135]:
email_data = email_data[email_data['Text'].str.strip().astype(bool)] # Удалим строки с пустыми ответами

# Очистка и токенизация текста

In [136]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

import contractions

from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\max_a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\max_a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
email_data['Text'] = email_data['Text'].str.lower() # Приведем символы к нижнему регистру
email_data['Text'] = email_data['Text'].apply(lambda x: contractions.fix(x)) # Преобразуем сокращения в полную запись: e.g. I'll -> I will
email_data['Text'] = email_data['Text'].apply(lambda x: re.sub('[\.\n\r\t]', ' ', x)) # Уберем символы переноса строк, табуляции
email_data['Text'] = email_data['Text'].apply(lambda x: re.sub('[А-Яа-я]', ' ', x))  # Уберем кириллицу
email_data['Text'] = email_data['Text'].apply(lambda x: re.sub(r'[\]!"$%&\'()*+,./:;=#@?\[\\^_`{|}~-’\d<>]+', " ", x)) # Удалим символы пунктуации

In [101]:
# Добавим колонку с токенизированным текстом (показалось, что правильнее будет токенизировать тексты по отдельности)
email_data['tokenized'] = email_data['Text'].apply(word_tokenize)

In [102]:
# Возможно, НЕ стоит очищать текст от stopwords, так как при этом, например, мы теряем конструкции со вспомогательным глаголом
# filtered_tokens = [word for word in tokens if not word in stopwords.words('english')]

In [103]:
tokens = list()
for tokens_list in email_data['tokenized']:
    tokens.extend(tokens_list)

for i in range(1,4):
    print(f'{i}-граммы')
    igrams = ngrams(tokens, i)
    icnt = Counter(igrams)
    n = 25
    print(f'{n} самых популярных')
    print(*icnt.most_common(25))
    print(f'Всего {icnt.total()}\n')
    

1-граммы
25 самых популярных
(('i',), 2538) (('to',), 1775) (('you',), 1439) (('the',), 1393) (('your',), 1302) (('is',), 1172) (('me',), 1108) (('my',), 1050) (('for',), 1050) (('a',), 966) (('in',), 899) (('and',), 815) (('about',), 789) (('it',), 766) (('have',), 756) (('of',), 535) (('that',), 505) (('with',), 485) (('am',), 479) (('what',), 466) (('from',), 451) (('email',), 416) (('dear',), 391) (('best',), 387) (('are',), 381)
Всего 54557

2-граммы
25 самых популярных
(('i', 'am'), 478) (('i', 'have'), 446) (('for', 'your'), 415) (('from', 'you'), 363) (('best', 'wishes'), 338) (('in', 'your'), 325) (('me', 'about'), 317) (('the', 'way'), 312) (('by', 'the'), 307) (('asked', 'me'), 293) (('you', 'asked'), 281) (('it', 'is'), 275) (('tell', 'me'), 255) (('more', 'about'), 254) (('about', 'your'), 247) (('way', 'tell'), 245) (('your', 'email'), 240) (('your', 'recent'), 234) (('me', 'more'), 233) (('you', 'in'), 226) (('to', 'get'), 225) (('am', 'always'), 210) (('that', 'is'), 19

In [104]:
# Соберем токены обратно в текст
email_data['prep_text'] = [' '.join(map(str, l)) for l in email_data['tokenized']]

# Кодирование текста

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [87]:
features = ['Question', 'prep_text']
targets = ['Solving a communicative task', 'Text structure', 'Use of English (for emails)']

test_size = int(0.25 * email_data.shape[0])  # Количество документов в тестовой выборке

# Разделим на тестовые и тренировочные данные
test_data_index = email_data.sample(test_size).index
train_data_index = email_data.index.difference(test_data_index)
test_data = email_data.loc[test_data_index]
train_data = email_data.loc[train_data_index]

In [105]:
def vectorize_as_bongrams(texts: list[str], ngram_min=1, ngram_max=3, max_words=12000):
    """
    Кодирует список текстов в список из закодированных векторов с помощью Bag of N-grams
    :param texts: Список текстов для кодирования
    :param ngram_min: Минимальная кратность n-граммы
    :param ngram_max: Максимальная кратность n-граммы
    :param max_words: Максимальное количество слов в словаре (размерность признакового пространства)
    :return: 
    """
    count_vectorizer = CountVectorizer(ngram_range=(ngram_min, ngram_max), max_features=max_words)
    emb = count_vectorizer.fit_transform(texts).toarray()
    return emb, count_vectorizer

In [106]:
# Закодируем в Bag of N-grams: от 1 до 3 -грамм 
train_encoded, train_vectorizer = vectorize_as_bongrams(train_data['prep_text'].to_list())
test_encoded = train_vectorizer.transform(test_data['prep_text'].to_list()).toarray() 

In [107]:
# Рассмотрим первые закодированные с помощью Bag of words слова 
print(train_encoded.shape)
pd.DataFrame(train_encoded.transpose(), index=train_vectorizer.get_feature_names_out()).head()

(303, 12000)


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able to,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able to do,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
about,1,1,0,1,1,2,4,4,2,1,...,1,0,1,1,1,2,2,2,2,2


In [108]:
def get_tfidf_matrix(texts: list[str], ngram_min=1, ngram_max=3, max_words=3000):
    tfidf_vectorizer = TfidfVectorizer(ngram_range = (ngram_min, ngram_max), max_features = max_words)
    emb = tfidf_vectorizer.fit_transform(texts).toarray()
    return emb, tfidf_vectorizer

In [109]:
# Закодируем слова с помощью tf-idf 
train_tfidf, train_tfidf_vectorizer = get_tfidf_matrix(train_data['prep_text'].to_list())
test_tfidf = train_tfidf_vectorizer.transform(test_data['prep_text'].to_list()).toarray() 

In [110]:
pd.DataFrame(train_tfidf.transpose(), index=train_tfidf_vectorizer.get_feature_names_out()).head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
able,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
able to,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
about,0.025747,0.025093,0.0,0.025582,0.023507,0.042626,0.085678,0.086063,0.035413,0.020578,...,0.021694,0.0,0.019373,0.023854,0.020672,0.041005,0.043803,0.045676,0.053066,0.044421
about arguments,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
about arguments with,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [127]:
feature_names = train_tfidf_vectorizer.get_feature_names_out()
pd.DataFrame(pd.concat([train_data['Question'].reset_index(), pd.DataFrame(feature_names[np.argmax(train_tfidf, axis=1)], columns=['Relevant word'])], axis=1))

,index,Question,Relevant word
0,2,…I am so happy that summer has come and we are...,visit
1,3,…I am learning to cook from my mother now. But...,cook
2,4,…I am so happy that summer has come and we are...,countries
3,5,…I’ve recently been involved in a school surve...,this is
4,6,…All of my friends think camping is a perfect ...,watch
...,...,...,...
298,533,... Have I ever told you that my dad’s hobby i...,extreme
299,551,questions about the film in Spanish,film
300,569,,trip to
301,572,,trip
